Лабораторная работа №7. "Полносвязные нейронные сети (многослойный персептрон). Решение задач регрессии и классификации"

Задание №1.
Решить задачи регрессии и классификации на данных в соответствии с Вашим индивидуальным вариантом (см. Лаб.работы №3, 4), используя полносвязные НС; реализовать НС посредством API Keras и фреймворка TensorFlow; оценить качество полученных моделей с помощью метрик.

Задание №2.
Разработать многослойный персептрон (MLP), с помощью которого можно решать задачи регрессии и классификации. Предусмотреть возможность использования таких функции активации, как sigmoid, tanh и relu; также предусмотреть возможность указать, сколько слоев нужно, сколько на каждом из них нейронов и какую функцию активации должен иметь слой. Реализовать обучение MLP методом обратного распространения ошибки; самостоятельно найти производные функций sigmoid, tanh и relu; реализовать классический градиентный спуск с возможностью указания шага.


Дополнительное Задание №3*.
1. Самостоятельно изучить отличия работы оптимизаторов Adam и RMSProp от классического градиентного спуска.
2. Реализовать градиентный спуск с использованием указанных оптимизаторов; предусмотрите возможность использования реализованных вами оптими-заторов в Вашем персептроне.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
import os

In [2]:
os.chdir('../data')
#Вариант 1
df_regression = pd.read_csv('lab3.csv')
df_classifier = pd.read_csv('lab4.csv')

In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

numeric_features_classifier = ['time','length'] 
df_classifier[numeric_features_classifier] = scaler.fit_transform(df_classifier[numeric_features_classifier])

In [4]:
from sklearn.model_selection import train_test_split
y_classifier = df_classifier["delay"]
X_classifier = df_classifier.drop(["delay"], axis=1)
X_classifier, _, y_classifier, _ = train_test_split(X_classifier, y_classifier, test_size=0.9, random_state=42)


y_regression = df_regression["price(euro)"]
X_regression = df_regression.drop(["price(euro)"], axis=1)

In [5]:
from imblearn.under_sampling import NearMiss
nm = NearMiss()
X_classifier, y_classifier = nm.fit_resample(X_classifier, y_classifier.ravel())

In [6]:
X_train_classifier, X_test_classifier, y_train_classifier, y_test_classifier = train_test_split(X_classifier, y_classifier, test_size=0.2)
X_train_regression, X_test_regression, y_train_regression, y_test_regression = train_test_split(X_regression, y_regression, test_size=0.2)

In [7]:
# для оценки качества решения задачи регрессии
from sklearn.metrics import mean_squared_error, mean_absolute_error
# для оценки качества решения задачи классификации
from sklearn.metrics import confusion_matrix, classification_report

In [8]:
import tensorflow as tf
import numpy as np

In [9]:
### Регрессия

In [10]:
# создаем модель, как набор последовательных слоев
model_regression = tf.keras.Sequential(
    [
        # Dense - полносвязный слой (каждый нейрон следующего слоя связан со всеми нейронами предыдущего)
        tf.keras.layers.Dense(64, activation="relu", input_shape=(939,)),
        # на втором скрытом слое будет 32 нейрона
        tf.keras.layers.Dense(32, activation="linear"),
        # Dropout позволяет внести фактор случайности - при обучении часть нейронов будет отключаться
        # каждый нейрон, в данном случае, будет отключаться с вероятностью 0.1
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        # на выходе один нейрон, функция активации не применяется
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [11]:
# посмотрим, какая сеть у нас получилась
model_regression.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                60160     
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 62785 (245.25 KB)
Trainable params: 62785 

In [12]:
# компилируем
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse")

In [13]:
X_train_regression = X_train_regression.astype('float32')
y_train_regression = y_train_regression.astype('float32')

In [14]:
# обучаем, 10 эпох означает 10 проходов по обучающей выборке
model_regression.fit(X_train_regression, y_train_regression, epochs=50, verbose=None)

In [15]:
X_test_regression = X_train_regression.astype('float32')
y_test_regression = y_train_regression.astype('float32')

In [16]:
y_pred_regression = model_regression.predict(X_test_regression)

752/752 [==============================] - 1s 830us/step


In [17]:
from sklearn.metrics import r2_score
# оцениваем качество с помощью метрик
print(mean_absolute_error(y_test_regression, y_pred_regression))
print(mean_squared_error(y_test_regression, y_pred_regression))
print(r2_score(y_test_regression, y_pred_regression))

2914.6333
15697916.0
0.4282546421316503


In [18]:
### Бинарная классификация

In [19]:
model_classification = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(608,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

In [20]:
X_train_classifier = X_train_classifier.astype('float32')
y_train_classifier = y_train_classifier.astype('float32')

In [21]:
# в качестве функции активации используется бинарная  кроссэнтропия
model_classification.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
# verbose=None - не будет логов
model_classification.fit(X_train_classifier, y_train_classifier, epochs=100, verbose=None)

In [22]:
X_test_classifier = X_test_classifier.astype('float32')
y_test_classifier = y_test_classifier.astype('float32')

In [23]:
# Предсказание на тестовых данных
y_pred_classifier = [np.argmax(pred) for pred in model_classification.predict(X_test_classifier, verbose=None)]
y_pred_classes = np.argmax(y_pred_classifier, axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test_classifier, y_pred_classifier)
print("Accuracy:", accuracy)

In [ ]:
print(classification_report(y_test_classifier, y_pred_classifier, zero_division=0))
print(confusion_matrix(y_test_classifier, y_pred_classifier))

Задание №2. Разработать многослойный персептрон (MLP), с помощью которого можно решать задачи регрессии и классификации. Предусмотреть возможность использования таких функции активации, как sigmoid, tanh и relu; также предусмотреть возможность указать, сколько слоев нужно, сколько на каждом из них нейронов и какую функцию активации должен иметь слой. Реализовать обучение MLP методом обратного распространения ошибки; самостоятельно найти производные функций sigmoid, tanh и relu; реализовать классический градиентный спуск с возможностью указания шага.

In [39]:
import numpy as np
from scipy.special import expit

class MLP:
    def __init__(self, input_size, hidden_layers, output_size, activations):
        """
        :param input_size: Размер входного слоя
        :param hidden_layers: Список, содержащий количество нейронов в каждом скрытом слое
        :param output_size: Размер выходного слоя
        :param activations: Список функций активации для каждого слоя
        :param learning_rate: Скорость обучения для градиентного спуска
        """
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.output_size = output_size
        self.activations = activations
        self.learning_rate = learning_rate

        # Инициализация весов и смещений
        self.weights, self.biases = self.initialize_parameters()

    def initialize_parameters(self):
        """
        Инициализация весов и смещений
        """
        sizes = [self.input_size] + self.hidden_layers + [self.output_size]
        weights = [np.random.randn(sizes[i], sizes[i+1]) for i in range(len(sizes)-1)]
        biases = [np.zeros((1, sizes[i+1])) for i in range(len(sizes)-1)]
        return weights, biases

    def sigmoid(self, x):
        """
        Сигмоидная функция активации
        """
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        """
        Производная сигмоидной функции
        """
        return self.sigmoid(x) * (1 - self.sigmoid(x))

    def tanh(self, x):
        """
        Гиперболический тангенс
        """
        return np.tanh(x)

    def tanh_derivative(self, x):
        """
        Производная гиперболического тангенса
        """
        return 1 - np.tanh(x)**2

    def relu(self, x):
        """
        Функция активации ReLU
        """
        return np.maximum(0, x)

    def relu_derivative(self, x):
        """
        Производная функции активации ReLU
        """
        return np.where(x > 0, 1, 0)

    def forward(self, X):
        """
        Прямое распространение (forward pass)
        """
        self.z_values = []
        self.activations_values = []

        # Входной слой
        a = X
        self.activations_values.append(a)

        # Скрытые слои
        for i in range(len(self.hidden_layers)):
            z = np.dot(a, self.weights[i]) + self.biases[i]
            self.z_values.append(z)
            a = self.activations[i](z)
            self.activations_values.append(a)

        # Выходной слой
        z = np.dot(a, self.weights[-1]) + self.biases[-1]
        self.z_values.append(z)
        a = self.sigmoid(z)  # Для задачи регрессии можно использовать другие функции активации
        self.activations_values.append(a)

        return a

    def backward(self, X, y):
        """
        Обратное распространение ошибки (backpropagation)
        """
        m = X.shape[0]
        delta = self.activations_values[-1] - y
        dz = delta * self.sigmoid_derivative(self.z_values[-1])
        dw = np.dot(self.activations_values[-2].T, dz) / m
        db = np.sum(dz, axis=0, keepdims=True) / m
        self.weights[-1] -= self.learning_rate * dw
        self.biases[-1] -= self.learning_rate * db

        for i in range(len(self.hidden_layers), 0, -1):
            delta = np.dot(dz, self.weights[i].T)
            dz = delta * self.activations[i-1](self.z_values[i-1])
            dw = np.dot(self.activations_values[i-2].T, dz) / m
            db = np.sum(dz, axis=0, keepdims=True) / m
            self.weights[i-1] -= self.learning_rate * dw
            self.biases[i-1] -= self.learning_rate * db

    def train(self, X, y, epochs):
        """
        Обучение модели
        """
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y)

            if epoch % 100 == 0:
                loss = np.mean(0.5 * (output - y)**2)
                print(f"Epoch {epoch}, Loss: {loss}")

    def predict(self, X):
        """
        Предсказание
        """
        return self.forward(X)

In [44]:
mlp_regressor = MLP(input_size=939, hidden_layers=[24050,939], output_size=1, activations=[np.tanh, expit])
mlp_regressor.train(X_train_regression, y_train_regression, epochs=50)

ValueError: Data must be 1-dimensional, got ndarray of shape (24050, 24050) instead

In [78]:
def find_max_cattle_population():
    initial_cattle_population = 74
    initial_wolf_population = 67
    wolf_shooting = 36
    final_wolf_population = 44
    final_cattle_population = 107

    # Расчет максимальной численности скота
    max_cattle_population = (final_cattle_population * (initial_wolf_population - wolf_shooting)) / (final_wolf_population - wolf_shooting)

    return int(max_cattle_population)

# Вызов функции и вывод результата
result = find_max_cattle_population()
print("Максимальное количество голов крупного рогатого скота за указанный период составляет: ", result)

Максимальное количество голов крупного рогатого скота за указанный период составляет:  414


In [75]:
import numpy as np
from scipy.constants import epsilon_0

# Дано
V0 = 5.0  # Напряжение на обкладках в вольтах
D = 0.4e-3  # Расстояние между обкладками в метрах
p0 = 1e4  # Объемная плотность заряда p0 в кл/м^5
e = 4.04e-12  # Диэлектрическая проницаемость в Ф/м
N = 1000  # Количество точек для аппроксимации

# Шаг по координате X
dx = D / N

# Инициализация массива для хранения значений потенциала
phi = np.zeros(N)

# Задание начальных и граничных условий
phi[0] = 0  # Потенциал на заземленной обкладке
phi[-1] = V0  # Потенциал на поданной напряжением обкладке

# Аппроксимация дифференциального уравнения методом конечных разностей
for i in range(1, N - 1):
    x = i * dx
    p_x = p0 * (x - D) ** 2
    phi[i + 1] = 2 * phi[i] - phi[i - 1] + (dx ** 2) * p_x / e

# Находим значение потенциала в центре конденсатора
phi_center = phi[N // 2]

# Вывод результата
print(f"Значение потенциала в центре конденсатора: {phi_center:.1f} В")

Значение потенциала в центре конденсатора: 5.6 В
